In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from dataset import get_dataset
import torchvision
import numpy as np
from arch import DQN_Conv

# Tải mô hình đơn giản (ví dụ: ResNet)
from arch import MNIST_CC

# 1. Cấu hình thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Tải mô hình đã huấn luyện trước
model = MNIST_CC()
model.load_state_dict(torch.load('trained_model\mnist_cc.pth'))

model.eval().to(device)  # Đặt chế độ đánh giá (evaluation mode)

# 3. Định nghĩa FGSM Attack
def fgsm_attack(image, epsilon, gradient):
    # Lấy dấu của gradient
    sign_gradient = gradient.sign()
    sign_gradient[sign_gradient < 0] = 0
    # Tạo mẫu tấn công
    perturbed_image = image + epsilon * sign_gradient
    # Giới hạn giá trị pixel [0, 1]
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    image_final = image + sign_gradient / torch.norm(sign_gradient, p=2) * 2
    image_final = torch.clamp(image_final, 0, 1)
    return image_final

# 4. Chuẩn bị dữ liệu
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
dataset = get_dataset('mnist', split='train')
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# 5. Tấn công mẫu đầu tiên
epsilon = 0.1  # Mức độ nhiễu
criterion = nn.CrossEntropyLoss()


dqn = DQN_Conv(28*28, 14*14)
dqn.load_state_dict(torch.load('model_0_trenvong_2.pth'))
dqn.eval().to(device)


<>:20: SyntaxWarning: invalid escape sequence '\m'
<>:20: SyntaxWarning: invalid escape sequence '\m'
C:\Users\hokha\AppData\Local\Temp\ipykernel_1520\966610712.py:20: SyntaxWarning: invalid escape sequence '\m'
  model.load_state_dict(torch.load('trained_model\mnist_cc.pth'))
C:\Users\hokha\AppData\Local\Temp\ipykernel_1520\966610712.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_g

DQN_Conv(
  (classifier): Sequential(
    (0): Conv2d(2, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.25, inplace=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=9216, out_features=1024, bias=True)
    (8): ReLU()
    (9): Dropout(p=0.5, inplace=False)
    (10): Linear(in_features=1024, out_features=128, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.5, inplace=False)
    (13): Linear(in_features=128, out_features=196, bias=True)
  )
)

In [9]:

t = 5

from tqdm import tqdm

total = 0
correct = 0

for images, labels in tqdm(dataloader):
    images, labels = images.to(device), labels.to(device)

    img_2_channel = torch.concatenate((images, images), 1)
    dqn_predict = dqn(img_2_channel)
    predict = dqn_predict.argmax(1)

    # dqn_img = dqn_predict.view(14, 14).detach().cpu().numpy()
    dqn_img = np.zeros((14, 14))
    dqn_img[predict//14, predict%14] = 1
    
    # Đặt chế độ tính gradient
    images.requires_grad = True
    
    # Dự đoán ban đầu
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Tính gradient
    model.zero_grad()
    loss.backward()
    gradient = images.grad.data

    grid_img = np.zeros((14, 14))
    for i in range(14):
        for j in range(14):
            grid_img[i, j] = gradient[0, 0, i*2:(i+1)*2, j*2:(j+1)*2].mean().item()

    # Tạo mẫu nhiễu
    perturbed_image = fgsm_attack(images, epsilon, gradient)

    distance = torch.norm(perturbed_image - images)
    print(f"Distance: {distance}")
    
    # Kiểm tra dự đoán trên mẫu bị tấn công
    outputs_perturbed = model(perturbed_image)
    _, final_pred = outputs_perturbed.max(1)
    
    # print(f"Label gốc: {labels.item()}, Dự đoán sau tấn công: {final_pred.item()}")

    # Hiển thị mẫu gốc, grad và mẫu bị tấn công
    # import matplotlib.pyplot as plt
    # plt.figure()
    # plt.subplot(141)
    # plt.imshow(images.squeeze().cpu().detach().numpy(), cmap='gray')
    # plt.title('Original Image')
    # plt.subplot(142)
    # plt.imshow(grid_img, cmap='gray')
    # plt.title('Gradient')
    # plt.subplot(143)
    # plt.imshow(perturbed_image.squeeze().detach().cpu().numpy(), cmap='gray')
    # plt.title('Perturbed Image')
    # plt.subplot(144)
    # plt.imshow(dqn_img, cmap='gray')
    # plt.title('DQN Image')


    # plt.show()
    
    # t -= 1
    # if t == 0:
    #     break
    total += 1
    if labels.item() != final_pred.item():
        correct += 1

    print(f"Accuracy: {correct/total}")

print(f"Accuracy: {correct/total}")

  0%|          | 4/60000 [00:00<55:37, 17.98it/s]

Distance: 1.9999998807907104
Accuracy: 1.0
Distance: 2.0
Accuracy: 1.0
Distance: 2.000000238418579
Accuracy: 0.6666666666666666
Distance: 1.9999998807907104
Accuracy: 0.5


  0%|          | 9/60000 [00:00<45:32, 21.96it/s]

Distance: 2.0
Accuracy: 0.6
Distance: 2.0
Accuracy: 0.6666666666666666
Distance: 2.0
Accuracy: 0.7142857142857143
Distance: 2.0
Accuracy: 0.75
Distance: 2.0
Accuracy: 0.6666666666666666


  0%|          | 12/60000 [00:00<41:53, 23.87it/s]

...

  1%|▏         | 859/60000 [00:30<37:23, 26.36it/s]

Distance: 2.0
Accuracy: 0.764018691588785
Distance: 2.0
Accuracy: 0.764294049008168
Distance: 1.999999761581421
Accuracy: 0.7634032634032634
Distance: 2.0
Accuracy: 0.7625145518044237
Distance: 2.0
Accuracy: 0.7627906976744186


  1%|▏         | 861/60000 [00:31<35:32, 27.73it/s]

Distance: 2.0
Accuracy: 0.7630662020905923


KeyboardInterrupt: 